# Mathematical Framework
### Model Assumptions:
1. Gene cassettes are in a plasmid and there are no long-range structural interactions.
2. There are no side reactions between species.
3. Geneandpromoterconcentrationisassumedtocontributeanegligibleimpactontheexpression
of the gene and are therefore modelled as constant.
1. Concentrations of RNAP and ribosome are relatively low.
2. The behaviour of the system can be approximated by the quasi-steady state (QSS) assumption.
3. The timescale of the system is short therefore dilution effects on mRNA can be absorbed into
the mRNA decay rate.
1. There is no degradation of protein (although it can be easily added to the dilution term).
2. Growth rate is constant to model the average rate of dilution.
3. Each promoter can only be activated or repressed by one TF of varying cooperativities.
4.  Combinatorial promoters exhibit competitive binding of TFs but only can only bind to one activator and one repressor (although it can be extended).
5.  There is assumed to be no ”leakiness” to minimise the number of parameters.

## Model Processes
This is a node, $i$. The 'gene' is a whole gene expression cassette which includes the actual gene sequence and regulatory sequences like the promoter, RBS and terminator.
<pre>
        1. Transcription       2. Translation  
               (TX)                 (TL)  
+------------+       +------------+       +------------+  
|            |       |            |       |            |  
|  Gene, g   +--+--> |   mRNA, m  +--+--> | Protein, x |  
|            |  |    |            |  |    |            |  
+------------+  |    +------------+  |    +------------+  
                |                    |  
              RNAP, [p]            Ribosome, [r]  
              + [ATP]              + [ATP]  
</pre>

- We will make use of the Catalyst.jl package to visualise chemical reaction equations

In [ ]:
using Catalyst, Latexify

## Modelling Transcription
Transcription from the gene, $g_i$ to the mRNA $m_i$.


In [6]:

node_1₁ = @reaction_network begin
    (k1⁺₁, k1⁻₁), g₁ + p <--> C₁ # Gene 1 binds with RNA polymerase to form the intermediate complex in a reversible reaction
    (k1θ₁), Cᵢ --> g₁ + p + m₁ # The intermediate complex reforms gene 1 and RNAP after transcription and mRNA 1 is formed
    (δ₁), m₁ --> ∅ # mRNA 1 decays at a rate δ₁
end

latexify(node_1)

L"\begin{align*}
\mathrm{g_1} + \mathrm{p} &\xrightleftharpoons[k1^-_1]{k1^+_1} \mathrm{C_1} \\
\mathrm{C_i} &\xrightarrow{k1\theta_1} \mathrm{g_1} + \mathrm{p} + \mathrm{m_1} \\
\mathrm{m_1} &\xrightarrow{\delta_1} \varnothing  
 \end{align*}
"

## Modelling Translation
Translation of the mRNA, $m_i$, to the protein $x_i$

In [7]:
node_1₂ = @reaction_network begin
    (k2⁺₁, k2⁻₁), m₁ + r <--> X₁ # mRNA 1 binds with a ribosome to form the intermediate complex in a reversible reaction
    (k2θ₁), C --> m₁ + r + x₁ # The intermediate complex reforms mRNA 1 and the ribosome after translation and protein 1 is formed
    (λ), x₁ --> ∅ # Protein 1 decays at a rate λ
end

Model ##ReactionSystem#337
States (5):
  m₂(t)
  p(t)
  X₂(t)
  C₂(t)
⋮
Parameters (4):
  k1⁺₂
  k1⁻₂
  k1θ₂
  λ

## Modelling Resource Coupling
Use of effective rate constants from [Characterisation and mitigation of gene expression burden in mammalian cells](https://www.nature.com/articles/s41467-020-18392-x) by Frei. *et al*.

Due to transcription and translation having a similar form, we can derive a net rate constant for both of them. First, we make some simplifications in notation:
\begin{align}
        k_{i}^+ &\in \{k_{1;i}^+, k_{2;i}^+\}\\
        k_{i}^- &\in \{k_{1;i}^-, k_{2;i}^-\}\\
        k_{i}^\theta &\in \{k_{1;i}^\theta, k_{2;i}^\theta\}
\end{align}

Now we define the binding constant:
\begin{align}
        k^m_i &= \frac{k_{i}^+}{k_{i}^- + k^{\theta}_{i}} \\
        k^m_i &\in \{k^m_{1;i}, k^m_{2;i}\}
\end{align}

With some derivations we find the effective rate constant:
\begin{align}
        k_{\text{eff}, i} &= \frac{k_{i}^{\theta} \cdot k^{m}_{i} \cdot R_T}{{1 + \sum_j k^{m}_{j} \cdot A_j}} \\
        k_{\text{eff}, i} &\in \{k_{\text{eff, 1};i} k_{\text{eff, 2};i}\}
\end{align}

This is the net rate of the production of product $B_i \in \{m_i, x_i\}$ from substrate $A_i \in \{g_i, m_i\}$

Therefore, transcription and translation reactions take the form:

In [12]:
eff_rxn = @reaction_network begin
    (k_eff), Aᵢ --> Bᵢ
    (δ), Aᵢ --> ∅
end

latexify(eff_rxn)

L"\begin{align*}
\mathrm{A_i} &\xrightarrow{k_{eff}} \mathrm{B_i} \\
\mathrm{A_i} &\xrightarrow{\delta} \varnothing  
 \end{align*}
"

## Modelling Regulatory Interactions
Regulatory interactions